In [4]:
#Importing necessary libraries:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
import random
import pyarrow.feather as feather

In [3]:
pip install pyarrow

     ---------------------------------------- 20.3/20.3 MB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
#Importing datasets:
covid_date = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\covid_date.feather")
fitbit_activities_fairly = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_activities_fairly.feather")
fitbit_activities_lightly = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_activities_lightly.feather")
fitbit_activities_sedentary = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_activities_sedentary.feather")
fitbit_activities_very = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_activities_very.feather")
fitbit_bmi = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_bmi.feather")
fitbit_calories = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_calories.feather")
fitbit_distance = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_distance.feather")
fitbit_hr = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_hr.feather")
fitbit_hrv = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_hrv.feather")
fitbit_sleep = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_sleep.feather")
fitbit_steps = pd.read_feather("C:\\AI Group Project\\UCL AI Project Covid Collab Dataset\\fitbit_steps.feather")

In [33]:
#Deleting unneeded coloumns:
fitbit_hr1 = fitbit_hr.loc[:,('user_id', 'dateTime', 'rhr')] 
fitbit_hrv1 = fitbit_hrv.loc[:,('user_id', 'datetime', 'dailyRMSSD')] 
fitbit_sleep1 = fitbit_sleep.loc[:,('user_id', 'date_of_sleep', 'efficiency', 'duration')]

In [34]:
#Preparing df for merging:
fitbit_hr1 = fitbit_hr1.rename(columns={'dateTime': 'datetime'})
fitbit_sleep1 = fitbit_sleep1.rename(columns={'date_of_sleep': 'datetime'}) 

In [ ]:
merged_fitbit = 

In [36]:
fitbit_dfs = [fitbit_hr1, fitbit_hrv1, fitbit_activities_fairly, fitbit_activities_lightly, fitbit_activities_sedentary, fitbit_activities_very,
              fitbit_bmi, fitbit_calories, fitbit_distance, fitbit_sleep1, fitbit_steps]

from functools import reduce

fitbit_merged = reduce(lambda left,right: pd.merge(left,right,on=['user_id', 'datetime'], how='outer', 
                sort=True), fitbit_dfs)

In [37]:
fitbit_merged.head()

,user_id,datetime,rhr,dailyRMSSD,activities_minutesFairlyActive,activities_minutesLightlyActive,activities_minutesSedentary,activities_minutesVeryActive,body_bmi,activities_calories,activities_distance,efficiency,duration,activities_steps
0,00AuZPNV0AdXvAzMf1ZkYy4nzrG3,2018-04-20,NaN,NaN,0.0,0.0,1440.0,0.0,20.482475,1488.0,0.00000,NaN,NaN,0.0
1,00AuZPNV0AdXvAzMf1ZkYy4nzrG3,2018-04-21,NaN,NaN,0.0,0.0,1440.0,0.0,20.482126,1486.0,0.00000,NaN,NaN,0.0
2,00AuZPNV0AdXvAzMf1ZkYy4nzrG3,2018-04-22,NaN,NaN,0.0,2.0,1438.0,0.0,20.481775,1490.0,0.05047,NaN,NaN,80.0
3,00AuZPNV0AdXvAzMf1ZkYy4nzrG3,2018-04-23,54.0,NaN,19.0,191.0,1207.0,23.0,20.481426,2240.0,5.84563,NaN,NaN,8339.0
4,00AuZPNV0AdXvAzMf1ZkYy4nzrG3,2018-04-24,53.0,NaN,74.0,279.0,603.0,60.0,20.481075,2930.0,10.92087,96.0,25440000.0,15579.0


In [38]:
#Merging "covid_date" with the merged fitbit dataframes:

dataframes = [covid_date, fitbit_merged]

from functools import reduce

all_merged = reduce(lambda left,right: pd.merge(left,right,on=['user_id'],
        how='left', sort=True), dataframes)

In [39]:
#Deleting unneeded coloums from the merged df:

#Changing datetime:
import datetime as datetime
all_merged["date_illness"] = pd.to_datetime(all_merged["date_illness"]).dt.date

In [40]:
#covid positive coloumn
all_merged.loc[all_merged["datetime"] >= all_merged["date_illness"], "covid_positive"] = "yes"
all_merged.loc[all_merged["datetime"] < all_merged["date_illness"], "covid_positive"] = "no"

In [41]:
all_merged.head()

,user_id,date_illness,datetime,rhr,dailyRMSSD,activities_minutesFairlyActive,activities_minutesLightlyActive,activities_minutesSedentary,activities_minutesVeryActive,body_bmi,activities_calories,activities_distance,efficiency,duration,activities_steps,covid_positive
0,00a9SPtBFkYF8KmWTvPPU0y69Xo1,2021-06-18,2017-02-02,77.0,NaN,0.0,38.0,1386.0,0.0,22.944088,1935.0,0.51915,NaN,NaN,684.0,no
1,00a9SPtBFkYF8KmWTvPPU0y69Xo1,2021-06-18,2017-02-03,69.0,NaN,10.0,287.0,734.0,5.0,22.793140,2906.0,6.36643,NaN,NaN,8333.0,no
2,00a9SPtBFkYF8KmWTvPPU0y69Xo1,2021-06-18,2017-02-04,64.0,NaN,36.0,237.0,619.0,103.0,22.642193,3740.0,13.60994,NaN,NaN,16150.0,no
3,00a9SPtBFkYF8KmWTvPPU0y69Xo1,2021-06-18,2017-02-05,60.0,NaN,0.0,201.0,607.0,0.0,22.823330,2511.0,3.10279,NaN,NaN,4088.0,no
4,00a9SPtBFkYF8KmWTvPPU0y69Xo1,2021-06-18,2017-02-06,57.0,NaN,47.0,183.0,750.0,120.0,23.004467,3783.0,8.61281,NaN,NaN,11468.0,no
